In [1]:
import copy

import pandas as pd
import numpy as np
import sklearn
import pickle
import datetime
import datatable as dtb
from datetime import datetime
import matplotlib.pyplot as plt

from quant_finance_research.utils import load_pickle, save_pickle
from quant_finance_research.fe.fe_val import panel_time_sumone_normalize_preprocess
from quant_finance_research.utils import get_example_df

In [2]:
data = load_pickle('../../../datasets/wind_factor/wind_and_barra_factor_insample.pkl')

In [11]:
data_org = load_pickle('../../../datasets/wind_factor/wind_factor_insample.pkl')

In [5]:
data.columns

Index(['YID', 'MicroSecondSinceEpoch', 'weight', 'next_1d', 'next_10d',
       'Industry', 'BETA', 'MOMENTUM', 'SIZE', 'EARNYILD',
       ...
       'TurnoverTechnicalFactor_s_tech_CYF',
       'TurnoverTechnicalFactor_s_tech_volumevolatility',
       'TurnoverTechnicalFactor_s_tech_volume1M60',
       'TurnoverTechnicalFactor_s_tech_volumn1M',
       'TurnoverTechnicalFactor_s_tech_turn5Dturn120',
       'TurnoverTechnicalFactor_s_tech_turn10Dturn120',
       'TurnoverTechnicalFactor_s_tech_turn20Dturn120',
       'TurnoverTechnicalFactor_s_tech_amount1m60',
       'TurnoverTechnicalFactor_S_TECH_CRY', 'positive'],
      dtype='object', length=162)

In [6]:
data_columns = data.columns.tolist()
data = data.reset_index()
del data['MicroSecondSinceEpoch'], data['positive']
data = data.rename(columns={'time': 'time_id', 'YID': 'investment_id'})
del data['id']

In [12]:
data_org_columns = data_org.columns.tolist()
data_org = data_org.reset_index()
del data_org['MicroSecondSinceEpoch'], data_org['positive']
data_org = data_org.rename(columns={'time': 'time_id', 'YID': 'investment_id'})
del data_org['id']

In [14]:
col_org = data_org.columns
for i in range(len(col_org)):
    print(col_org[i])

time_id
investment_id
weight
next_1d
next_10d
DailyValuationFactor_S_DFA_LNMV
DailyValuationFactor_S_DFA_LNTOTASSETS
DailyValuationFactor_S_DFA_FLOATMV
DailyValuationFactor_S_DFA_LNFLOATMV
DailyValuationFactor_S_DFA_OPPS
DailyValuationFactor_s_dfa_pettm_deducted
DailyValuationFactor_S_DFA_MVTOEBITDA
DailyValuationFactor_S_DFA_PROFITTOMV
ConsensusExpectationFactor_s_west_netprofit_ftm_chg_1m
ConsensusExpectationFactor_s_west_netprofit_ftm_chg_3m
ConsensusExpectationFactor_s_west_netprofit_ftm_chg_6m
ConsensusExpectationFactor_s_west_eps_ftm_chg_1m
ConsensusExpectationFactor_s_west_eps_ftm_chg_3m
ConsensusExpectationFactor_s_west_eps_ftm_chg_6m
ConsensusExpectationFactor_s_west_sales_ftm_chg_1m
ConsensusExpectationFactor_s_west_sales_ftm_chg_3m
ConsensusExpectationFactor_s_west_sales_ftm_chg_6m
ConsensusExpectationFactor_s_west_netprofit_fy1_chg_1m
ConsensusExpectationFactor_s_west_netprofit_fy1_chg_3m
ConsensusExpectationFactor_s_west_netprofit_fy1_chg_6m
ConsensusExpectationFactor_s_we

In [10]:
col = data.columns
for i in range(len(col)):
    print(col[i])

time_id
investment_id
weight
next_1d
next_10d
Industry
BETA
MOMENTUM
SIZE
EARNYILD
RESVOL
GROWTH
BTOP
LEVERAGE
LIQUIDTY
SIZENL
DailyValuationFactor_S_DFA_LNMV
DailyValuationFactor_S_DFA_LNTOTASSETS
DailyValuationFactor_S_DFA_FLOATMV
DailyValuationFactor_S_DFA_LNFLOATMV
DailyValuationFactor_S_DFA_OPPS
DailyValuationFactor_s_dfa_pettm_deducted
DailyValuationFactor_S_DFA_MVTOEBITDA
DailyValuationFactor_S_DFA_PROFITTOMV
ConsensusExpectationFactor_s_west_netprofit_ftm_chg_1m
ConsensusExpectationFactor_s_west_netprofit_ftm_chg_3m
ConsensusExpectationFactor_s_west_netprofit_ftm_chg_6m
ConsensusExpectationFactor_s_west_eps_ftm_chg_1m
ConsensusExpectationFactor_s_west_eps_ftm_chg_3m
ConsensusExpectationFactor_s_west_eps_ftm_chg_6m
ConsensusExpectationFactor_s_west_sales_ftm_chg_1m
ConsensusExpectationFactor_s_west_sales_ftm_chg_3m
ConsensusExpectationFactor_s_west_sales_ftm_chg_6m
ConsensusExpectationFactor_s_west_netprofit_fy1_chg_1m
ConsensusExpectationFactor_s_west_netprofit_fy1_chg_3m
Conse

In [8]:
data.shape

(6553814, 160)

In [17]:
new_feat = [col[i] for i in range(len(col[i])) if col[i] not in col_org]
new_feat_idx = [i for i in range(len(col[i])) if col[i] in new_feat]

In [16]:
new_feat

['Industry',
 'BETA',
 'MOMENTUM',
 'SIZE',
 'EARNYILD',
 'RESVOL',
 'GROWTH',
 'BTOP',
 'LEVERAGE',
 'LIQUIDTY',
 'SIZENL']

In [18]:
new_feat_idx

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [ ]:

def get_wind_data(out_sample_start, file='no_barra'):
    if file == 'no_barra':
        data = load_pickle('../../../datasets/wind_factor/wind_factor_insample.pkl')
    elif file == 'barra':
        data = load_pickle('../../../datasets/wind_factor/wind_and_barra_factor_insample.pkl')
    else:
        raise ValueError(f"No such file type: {file}")
    print(data.shape)
    # data = data.iloc[:10000, :]

    data_columns = data.columns.tolist()

    data = data.reset_index()
    del data['MicroSecondSinceEpoch'], data['positive']
    data = data.rename(columns={'time': 'time_id', 'YID': 'investment_id'})
    del data['id']

    x_column = [i for i in range(5, data.shape[1])]
    loss_column = [2]
    y_column = [3, 4]
    df_column = {'x': x_column, 'y': y_column, 'loss': loss_column}
    data = data.dropna(axis=0, how='any')
    
    if file == 'barra':
        

    insample = data[data['time_id'] < out_sample_start]
    outsample = data[data['time_id'] >= out_sample_start]
    set_df_continue_index(insample)
    set_df_continue_index(outsample)

    insample, _ = panel_time_sumone_normalize_preprocess(insample, df_column, df_column['loss'], time_col='time_id')
    outsample, _ = panel_time_sumone_normalize_preprocess(outsample, df_column, df_column['loss'], time_col='time_id')

    return insample, outsample, df_column, [3, 4]